## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import os
import sys

sys.path.insert(1, os.path.join("..","."))

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Broome,US,42.2506,-75.8330,70.02,28,0,8.05,clear sky
1,1,Kuryk,KZ,43.2000,51.6500,68.29,46,6,18.66,clear sky
2,2,Atuona,PF,-9.8000,-139.0333,78.89,77,65,13.65,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,57.20,84,9,2.64,clear sky
4,4,Bandarbeyla,SO,9.4942,50.8122,84.47,76,64,10.42,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp) ]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Atuona,PF,-9.8000,-139.0333,78.89,77,65,13.65,broken clouds
4,4,Bandarbeyla,SO,9.4942,50.8122,84.47,76,64,10.42,broken clouds
18,18,Vaini,TO,-21.2000,-175.2000,78.96,94,75,9.22,broken clouds
42,42,Honiara,SB,-9.4333,159.9500,77.50,86,100,1.03,moderate rain
62,62,Stephenville,US,32.2207,-98.2023,79.81,34,75,16.11,broken clouds


In [15]:
# 4a. Determine if there are any empty rows.
# Different ways to check and no null values exist

preferred_cities_df.info()

print(preferred_cities_df.count())

print(preferred_cities_df.notnull().sum())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 2 to 750
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              220 non-null    int64  
 1   City                 220 non-null    object 
 2   Country              220 non-null    object 
 3   Lat                  220 non-null    float64
 4   Lng                  220 non-null    float64
 5   Max Temp             220 non-null    float64
 6   Humidity             220 non-null    int64  
 7   Cloudiness           220 non-null    int64  
 8   Wind Speed           220 non-null    float64
 9   Current Description  220 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 18.9+ KB
City_ID                220
City                   220
Country                220
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed      

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No null rows, putting logic for future use case
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Atuona,PF,-9.8000,-139.0333,78.89,77,65,13.65,broken clouds
4,4,Bandarbeyla,SO,9.4942,50.8122,84.47,76,64,10.42,broken clouds
18,18,Vaini,TO,-21.2000,-175.2000,78.96,94,75,9.22,broken clouds
42,42,Honiara,SB,-9.4333,159.9500,77.50,86,100,1.03,moderate rain
62,62,Stephenville,US,32.2207,-98.2023,79.81,34,75,16.11,broken clouds


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.89,broken clouds,-9.8000,-139.0333,
4,Bandarbeyla,SO,84.47,broken clouds,9.4942,50.8122,
18,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
42,Honiara,SB,77.50,moderate rain,-9.4333,159.9500,
62,Stephenville,US,79.81,broken clouds,32.2207,-98.2023,
66,Coruripe,BR,82.71,broken clouds,-10.1256,-36.1756,
73,Saint-Philippe,RE,76.59,scattered clouds,-21.3585,55.7679,
80,Saint George,US,78.80,clear sky,37.1041,-113.5841,
84,Talara,PE,79.56,clear sky,-4.5772,-81.2719,
88,Lhokseumawe,ID,80.26,overcast clouds,5.1801,97.1507,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print(f'Hotel Not Found in {row["City"]}... Skipping')    
        

Hotel Not Found in Jutai... Skipping
Hotel Not Found in Qandala... Skipping
Hotel Not Found in Poum... Skipping
Hotel Not Found in Tacuati... Skipping
Hotel Not Found in Merritt Island... Skipping
Hotel Not Found in Benin... Skipping
Hotel Not Found in Thinadhoo... Skipping
Hotel Not Found in Nicolas Bravo... Skipping
Hotel Not Found in Sembe... Skipping


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.89,broken clouds,-9.8000,-139.0333,Villa Enata
4,Bandarbeyla,SO,84.47,broken clouds,9.4942,50.8122,JABIR HOTEL
18,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
42,Honiara,SB,77.50,moderate rain,-9.4333,159.9500,Heritage Park Hotel
62,Stephenville,US,79.81,broken clouds,32.2207,-98.2023,La Quinta Inn & Suites by Wyndham Stephenville


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))